In [1]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [2]:
# Lineitem

l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX",
                "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT", "L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()

l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../data_storage/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

# Order

o_columnnames = ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"]

for i in range(len(o_columnnames)):
    o_columnnames[i] = o_columnnames[i].lower()
    
o_data_types = {
    'o_orderkey': int,
    'o_custkey': int,
    'o_orderstatus': str,
    'o_totalprice': float,
    'o_orderpriority': str,
    'o_clerk': str,
    'o_shippriority': int,
    'o_comment': str
}

o_parse_dates = ['o_orderdate']

# Don't set indexes, as we can't access them with Pandas selection!
orders = pd.read_table("../data_storage/orders.tbl.csv", sep="|", names=o_columnnames, dtype=o_data_types, parse_dates=o_parse_dates)

# Customer

c_columnnames = ["C_CUSTKEY", "C_NAME", "C_ADDRESS", "C_NATIONKEY", "C_PHONE", "C_ACCTBAL", "C_MKTSEGMENT", "C_COMMENT"]

for i in range(len(c_columnnames)):
    c_columnnames[i] = c_columnnames[i].lower()
    
c_data_types = {
    'c_custkey': int,
    'c_name': str,
    'c_address': str,
    'c_nationkey': int,
    'c_phone': str,
    'c_acctbal': float,
    'c_mktsegment': str,
    'c_comment': str
}

c_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
customer = pd.read_table("../data_storage/customer.tbl.csv", sep="|", names=c_columnnames, dtype=c_data_types, parse_dates=c_parse_dates)

# Part

p_columnnames = ["P_PARTKEY", "P_NAME", "P_MFGR", "P_BRAND", "P_TYPE", "P_SIZE", "P_CONTAINER", "P_RETAILPRICE", "P_COMMENT"]

for i in range(len(p_columnnames)):
    p_columnnames[i] = p_columnnames[i].lower()
    
p_data_types = {
    'p_partkey': int, 
    'p_name': str,
    'p_mfgr': str,
    'p_brand': str,
    'p_type': str,
    'p_size': int,
    'p_container': str,
    'p_retailprice': float,
    'p_comment': str
}

p_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
part = pd.read_table("../data_storage/part.tbl.csv", sep="|", names=p_columnnames, dtype=p_data_types, parse_dates=p_parse_dates)

# Nation

n_columnnames = ["N_NATIONKEY", "N_NAME", "N_REGIONKEY", "N_COMMENT"]

for i in range(len(n_columnnames)):
    n_columnnames[i] = n_columnnames[i].lower()
    
n_data_types = {
    'n_nationkey': int,
    'n_name': str,
    'n_regionkey': int,
    'n_comment': str,
}

n_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
nation = pd.read_table("../data_storage/nation.tbl.csv", sep="|", names=n_columnnames, dtype=n_data_types, parse_dates=n_parse_dates)

# Supplier

s_columnnames = ["S_SUPPKEY", "S_NAME", "S_ADDRESS", "S_NATIONKEY", "S_PHONE", "S_ACCTBAL", "S_COMMENT"]

for i in range(len(s_columnnames)):
    s_columnnames[i] = s_columnnames[i].lower()

s_data_types = {
    's_suppkey': int,
    's_name': str,
    's_address': str,
    's_nationkey': int,
    's_phone': str,
    's_acctbal': float,
    's_comment': str
}

s_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
# Don't set indexes, as we can't access them with Pandas selection!
supplier = pd.read_table("../data_storage/supplier.tbl.csv", sep="|", names=s_columnnames, dtype=s_data_types, parse_dates=s_parse_dates)

In [3]:
df_filter_1 = nation[nation.n_name == "SAUDI ARABIA"]
df_filter_2 = supplier
df_merge_1 = df_filter_2.merge(df_filter_1, left_on='s_nationkey', right_on='n_nationkey')
# Correct

In [4]:
df_filter_3 = lineitem[lineitem.l_receiptdate > lineitem.l_commitdate]
df_merge_2 = df_filter_3.merge(df_merge_1, left_on='l_suppkey', right_on='s_suppkey')
# Correct

In [24]:
tmp = df_filter_3.groupby(['l_orderkey'], sort=False, as_index=False)['l_suppkey'].nunique()

In [25]:
tmp = tmp[tmp['l_suppkey'] == 1]

joined = df_merge_2.merge(tmp, left_on='l_orderkey', right_on='l_orderkey')
joined

,l_orderkey,l_partkey,l_suppkey_x,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,...,s_address,s_nationkey,s_phone,s_acctbal,s_comment,n_nationkey,n_name,n_regionkey,n_comment,l_suppkey_y
0,166305,114000,1534,1,36.0,36504.0,0.09,0.06,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
1,528099,6533,1534,2,30.0,43185.9,0.01,0.08,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
2,602727,149019,1534,2,13.0,13884.13,0.09,0.06,A,F,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
3,672642,151533,1534,1,1.0,1584.53,0.02,0.07,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
4,882917,49029,1534,1,8.0,7824.16,0.01,0.03,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13854,5003525,197661,7662,1,48.0,84415.68,0.09,0.03,R,F,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
13855,5078821,192623,7662,1,48.0,82349.76,0.04,0.02,N,O,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
13856,5614853,65155,7662,1,29.0,32484.35,0.09,0.05,R,F,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1
13857,5617154,60143,7662,1,10.0,11031.4,0.01,0.05,N,O,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,1


In [29]:
df_filter_4 = lineitem[lineitem.l_receiptdate > lineitem.l_commitdate]



df_merge_3 = df_merge_2.merge(df_filter_4, left_on='l_orderkey', right_on='l_orderkey', how='inner') # , indicator=True
#print(df_merge_3.columns)
# print(df_merge_3._merge.value_counts())
df_merge_3 = df_merge_3[df_merge_3.l_suppkey_x != df_merge_3.l_suppkey_y]['l_orderkey']
# df_merge_3 = df_merge_3[df_merge_3._merge == "left_only"]


test = df_merge_2.merge(df_merge_3, left_on='l_orderkey', right_on='l_orderkey', how='outer', indicator=True)
test = test[test._merge == "left_only"]

In [30]:
test

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,...,s_address,s_nationkey,s_phone,s_acctbal,s_comment,n_nationkey,n_name,n_regionkey,n_comment,_merge
30,166305,114000,1534,1,36.0,36504.0,0.09,0.06,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
114,528099,6533,1534,2,30.0,43185.9,0.01,0.08,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
122,602727,149019,1534,2,13.0,13884.13,0.09,0.06,A,F,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
127,672642,151533,1534,1,1.0,1584.53,0.02,0.07,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
159,882917,49029,1534,1,8.0,7824.16,0.01,0.03,N,O,...,",rRNXQRJQd6JXzpwnEuKkKn",20,30-891-630-2295,3863.61,ounts believe around the f,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465177,5003525,197661,7662,1,48.0,84415.68,0.09,0.03,R,F,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
465197,5078821,192623,7662,1,48.0,82349.76,0.04,0.02,N,O,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
465272,5614853,65155,7662,1,29.0,32484.35,0.09,0.05,R,F,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only
465277,5617154,60143,7662,1,10.0,11031.4,0.01,0.05,N,O,...,"VGk9AgllnOQ2,yPPk,pfHXJ4NgF2",20,30-162-843-3198,5734.03,arefully express deposits. carefully,20,SAUDI ARABIA,4,ts. silent requests haggle. closely express pa...,left_only


In [10]:
df_merge_3

,l_orderkey,l_partkey_x,l_suppkey_x,l_linenumber_x,l_quantity_x,l_extendedprice_x,l_discount_x,l_tax_x,l_returnflag_x,l_linestatus_x,...,l_tax_y,l_returnflag_y,l_linestatus_y,l_shipdate_y,l_commitdate_y,l_receiptdate_y,l_shipinstruct_y,l_shipmode_y,l_comment_y,_merge
